# Clase 13: Detección de Anomalías y Visualización en Baja Dimensionalidad

**MDS7202: Laboratorio de Programación Científica para Ciencia de Datos**




## Objetivos de la Clase

- Entender qué es un outlier y explorar distintas téncicas para encontrarlos.
- Explorar técnicas para visualizar datos en baja dimensionalidad.

## Detección y manejo de Anomalías

> **Pregunta**: ¿Qué es una anomalía?

Una anomalía (*outlier* en ingles) es un dato significativamente distinto a los demás. Se puede considerar como una observación cuya desviación del conjunto de datos permite establecer la hipótesis, de que su generación fue obtenida por un mecanismo distinto al principal en la modelación de un fenómeno.

![Outlier](../../recursos/2023-01/16-Detección-anomalias/outliers.gif)

> **Pregunta**: ¿Por qué debería preocuparme por estos valores?

Las anomalías contienen por tanto información sobre características anormales de las entidades y esquemas que impactan el proceso generativo de los datos. Reconocer estas observaciones permite:
- Desde el punto de vista teórico, mejorar el entendimiento de los problemas modelados. 
- Desde el punto de vista práctico, permite mejorar procesos de adquisición de datos y presición de modelos.


Antes de continuar, es necesario hacer una distinción entre términos:

- **Detección de Outliers** : Detectamos anomalías sobre los datos que estamos analizando o sobre los datos de entrenamiento del modelo.

- **Novelty Detection**: Cuando detectamos outliers sobre **datos nuevos**.


## Tipos de Outliers

- **Univariados**: Solo en una característica de los datos.
- **Multivariados**: Outlier al combinar más de una característica de los datos.
   

## Origen de los Outliers


> **Pregunta ❓**: ¿Cuáles son las posibles fuentes de Outliers?


- Errores al transcribir los Datos.
- Errores de Medición.
- Errores Experimentales.
- Errores del preprocesamiento.
- Al extraer o mezclar datos que no son compatibles entre si.
- Naturales.

## Dataset de Hoy: Wine Quality 🍷

<div style="text-align: center;">
    <img src="../../recursos/2023-01/16-Detección-anomalias/wine.jpg" style="width: 50%;">
</div>

<center> Fuente de la imagen: https://www.baysidegroup.com.au/blog/could-native-grapes-be-the-future-of-australian-wine/</center>

Wine Quality es un dataset de características que describen vinos portugeses de la variedad "Vinho Verde".

Atributos:

    1 - fixed acidity
    2 - volatile acidity
    3 - citric acid
    4 - residual sugar
    5 - chlorides
    6 - free sulfur dioxide
    7 - total sulfur dioxide
    8 - density
    9 - pH
    10 - sulphates
    11 - alcohol
    
Output: Calidad subjetiva del vino.

    12 - quality (score between 0 and 10)

In [ ]:
import pandas as pd
import plotly.express as px

df = pd.read_csv("../../recursos/2023-01/16-Detección-anomalias/wineQualityReds.csv", index_col=0)
df.head()

In [ ]:
df.describe()

In [ ]:
for col in df:
    fig = px.histogram(df, x=col, marginal="box", title = 'Histogram of ' + col)
    fig.show()

## Métodos de Manejo de Anomalías Univariados


#### Desviación Estándar

Si se estima que la variable a estudiar se distribuye de manera normal, entonces el 95% de los datos se encuentra a 2 desviaciones estándar de la media, mientras que el 99.7% se encuentra dentro de 3 desviaciones estándar. Basándose en esto, se puede considerar que cualquier punto fuera de 3 desviaciones estándar de la media como candidato a anomalía. Una forma más flexible de estimar anomalías usando el principio de normalidad, es por medio de z-scores. 


$$\text{z-score} = \frac{x_i - \overline{x}}{s}$$


<div style="text-align: center;">
    <img src="../../recursos/2023-01/16-Detección-anomalias/norm.png" style="width: 75%;">
</div>

In [ ]:
import plotly.figure_factory as ff

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(
    [df["fixed.acidity"].values], ["fixed.acidity"], curve_type="normal", show_rug=False
)
fig.show()

In [ ]:
from scipy.stats import zscore

z_scores = zscore(df["fixed.acidity"])
z_scores

In [ ]:
# agregamos los zscores al df
df["fixed.acidity_zscores"] = z_scores
df

In [ ]:
df["fixed.acidity_zscores"].abs() > 3

In [ ]:
# agregamos un booleano por cada fila que indica si la observación está
# a 3 o más distribuciones estandar entonces es booleano.
df["fixed.acidity_outlier"] = df["fixed.acidity_zscores"].abs() > 3

df[["fixed.acidity", "fixed.acidity_zscores", "fixed.acidity_outlier"]].head(10)

In [ ]:
df.loc[
    df["fixed.acidity_outlier"], # condicion: fixed.acidity_outlier == True
    ["fixed.acidity", "fixed.acidity_zscores", "fixed.acidity_outlier"],
]

In [ ]:
fig = px.histogram(df, x="fixed.acidity", color="fixed.acidity_outlier")
fig.update_layout(showlegend=False)

Si se va a estudiar una columna con ouliers mediante este método, es coveniente hacer un **test de normalidad**. Si la variable no cumple con la hipótesis, es posible preprocesarla usando el método de **box-cox**, **Yeo-Johnson** o **Inter Quantilico**. Se recomienda este último por ser robusto a outliers. 


#### IQR: Rango Intercuartilico

El rango intercuartílico (**IQR**) se utiliza para medir la dispersión de los datos. 
Para obtenerlo: 

Los datos se separan en 4 grupo de (casi) igual tamaño.
El IQR se calcula como la diferencia entre el primer cuartil $Q1$ (25%) y el tercero $Q3$ (75%) : $IQR = Q3 - Q1$.

<div align='center'>
    <img src="https://miro.medium.com/max/9000/1*2c21SkzJMf3frPXPAR_gZA.png" style="width: 50%;">
    <p> 
        Fuente: 
        <a href='https://www.kdnuggets.com/2019/11/understanding-boxplots.html'>Understanding Boxplots
        </a> 
    </p>
</div>

In [ ]:
df["total.sulfur.dioxide"].describe()

In [ ]:
px.histogram(df, "total.sulfur.dioxide", marginal="box")

In [ ]:
desc = df["total.sulfur.dioxide"].describe()
desc

In [ ]:
iqr = desc["75%"] - desc["25%"]
iqr

Luego, los valores que estén en $25\% - (IQR * 1.5)$

In [ ]:
cota_inf = desc["25%"] - iqr * 1.5
cota_inf

Y los valores que estén sobre $75\% + (IQR * 1.5)$

In [ ]:
cota_sup = desc["75%"] + iqr * 1.5
cota_sup

Pueden ser considerados como outliers

In [ ]:
df["total.sulfur.dioxide_outlier"] = (
    df["total.sulfur.dioxide"] > cota_sup) | (
    df["total.sulfur.dioxide"] < cota_inf
)

In [ ]:
import plotly.express as px

fig1 = px.box(df, x="total.sulfur.dioxide", height=200)
fig1.show()

fig2 = px.histogram(
    df,
    x="total.sulfur.dioxide",
    color="total.sulfur.dioxide_outlier",
    title="total.sulfur.dioxide<br><sup>Rojo = outlier</sup>",
)
fig2.update_layout(showlegend=False)
fig2.show()

En el gráfico de caja, vemos que los outliers están sobre y bajo las lineas rectas, cada una representa Q1 - 1.5 x IQR (linea inferior) y Q3 + 1.5 x IQR (linea superior) Los valores dentro de la caja corresponden al IQR y la linea central es la mediana de los datos.

> **Pregunta:** ¿Por qué 1.5 y no otro número?

<div style="text-align: center;">
    <img src="../../recursos/2023-01/16-Detección-anomalias/iqr_origen.png" style="width: 50%;">
</div>

## Reducción de Dimensionalidad para la Visualización 

Como hemos visto en las clases anteriores, todos las observaciones se componen de muchas variables/características distintas. Mientras más características se tengan, más complejo se torna el análisis exploratorio de datos, la detección de outliers y la creación de modelos predictivos.



In [ ]:
df.head()



Por lo tanto, sería de mucha utilidad contar con mecanismos para reducir la cantidad de características. Existen 2 enfoques: 

- **Selección de Atributos Relevantes**
- **Reducción de Dimensionalidad.**

Ambos serán vistos con mayor profundidad más adelante. 

Sin embargo, existe un par de métodos de reducción de dimensionalidad que nos pueden ser particularmente útiles para la tarea que estamos resolviendo en este momento (detección de outliers) y en general, para crear una idea de como se distribuyen los datos al considerar todas las características.


Estos son los **métodos de reducción de dimensionalidad para visualización**. Estas son técnicas que nos permiten representar cada observación (un vector de alta dimensionalidad) en un vector de dos o tres dimensiones (mucho más sencillo para graficar).


**Paréntesis: Manifold learning**  


> Manifold learning es un enfoque para la reducción de dimensionalidad no lineal. Los algoritmos para esta tarea se basan en la idea de que la dimensionalidad de muchos conjuntos de datos es solo artificialmente alta.

### t-distributed Stochastic Neighbor Embedding (t-SNE)


La idea detrás de este método es proveer un método efectivo de reducción de dimensionalidad para visualizar un dataset complejo. El objetivo es conservar la mayor parte de la información en la dimensión baja a la vez que permita visualizar clusters y estructura general de los datos.

Este método conserva la localidad de las observaciones: Es decir, las observaciones similares queden en vectores cercanos y observaciones distintas en vectores lejanos (cercanos y lejanos en función de sus distancias).


Sin embargo, el método posee un par de desventajas:

- Es computacionalmente caro. Puede tomar horas transformar más de un millon de datos.
- Es estocástico: diferentes ejecuciones entregan distintas proyecciones.
- Las relaciones entre distancias globales no se preservan correctamente.

> **Nota**: Es importante tener a la misma escala todos los datos.

<div style="text-align: center;">
    <img src="https://miro.medium.com/max/800/1*lKLB_1aghhnxQjMQziEyGQ.gif" style="width: 50%;">
</div>

<center>Fuente de la animación: https://www.oreilly.com/content/an-illustrated-introduction-to-the-t-sne-algorithm/</center>


No veremos la implementación de este método en clases, pero pueden encontrar una muy buena referencia en el siguiente video: https://www.youtube.com/watch?v=NEaUSP4YerM

Ver también: **https://projector.tensorflow.org/**

In [ ]:
features = df.drop(
    columns=[
        "quality",
        "fixed.acidity_zscores",
        "fixed.acidity_outlier",
        "total.sulfur.dioxide_outlier",
    ]
)
quality = df.loc[:, ["quality"]]

features.head(3)

In [ ]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler

# Notar que hacemos un escalamiento previo!!
scaled_features = MinMaxScaler().fit_transform(features)

# Creamos una instancia de tsne
tsne = TSNE(n_components=2, random_state=42)

# Transformamos los datos
wine_features_tsne_embedded = tsne.fit_transform(scaled_features)
wine_features_tsne_embedded

In [ ]:
# Noten que guardamos la transformación en df y no en features.
df["x_tsne"] = wine_features_tsne_embedded[:, 0]
df["y_tsne"] = wine_features_tsne_embedded[:, 1]

In [ ]:
px.scatter(df, x="x_tsne", y="y_tsne")

In [ ]:
px.scatter(df, x="x_tsne", y="y_tsne", color="pH")

In [ ]:
px.scatter(df, x="x_tsne", y="y_tsne", color="alcohol")

In [ ]:
px.scatter(df, x="x_tsne", y="y_tsne", color="residual.sugar")

In [ ]:
px.scatter(df, x="x_tsne", y="y_tsne", color="quality")

In [ ]:
px.scatter(df, x="x_tsne", y="y_tsne", color="fixed.acidity")

In [ ]:
# probamos el criterio de outliers z-score
px.scatter(df, x="x_tsne", y="y_tsne", color="fixed.acidity_outlier")

### Uniform Manifold Approximation and Projection (UMAP)

Relativamente novedoso método de reducción de dimensionalidad. Puede ser usado tanto para visualización como para reducción de dimensionalidad para modelos predictivos. 

Presenta varias mejoras que con respecto a TSNE.  

- Es más rápido y acepta una mayor cantidad de datos. 
- Conserva las distancias globales entre puntos.

In [ ]:
# hay que instalarlo aparte.
!pip install umap-learn

In [ ]:
import umap

# Notar que hacemos un escalamiento previo!!
scaled_features = MinMaxScaler().fit_transform(features)

umap = umap.UMAP()

wine_features_umap_embedded = umap.fit_transform(scaled_features)
wine_features_umap_embedded

In [ ]:
# Noten que guardamos la transformación en df y no en features.
df["x_umap"] = wine_features_umap_embedded[:, 0]
df["y_umap"] = wine_features_umap_embedded[:, 1]

In [ ]:
from plotly.subplots import make_subplots

fig1 = px.scatter(
    df,
    x="x_umap",
    y="y_umap",
)

fig2 = px.scatter(
    df,
    x="x_tsne",
    y="y_tsne",
)

# izquierda: UMAP
# derecha: TSNE
fig = (
    make_subplots(rows=1, cols=2)
    .add_trace(fig1.data[0], row=1, col=1)
    .add_trace(fig2.data[0], row=1, col=2)
    .update_layout(height=400, title_text="Comparación UMAP - TSNE")
    .show()
)

In [ ]:
px.scatter(
    df,
    x="x_umap",
    y="y_umap",
    color="quality",
)

In [ ]:
px.scatter(df, x="x_umap", y="y_umap", color="fixed.acidity_outlier")

In [ ]:
px.scatter(df, x="x_umap", y="y_umap", color="fixed.acidity")

> **Pregunta:** ¿Son suficientes los detectores de outliers sobre sola una variable?

## Métodos de Manejo de Outliers Multivariados


Este tipo de métodos permiten encontrar outliers considerando no solo un atributo en particular, si no que todos los atributos al mismo tiempo.

### Paréntesis: Clustering


**El Clustering es la tarea de agrupar distintas observaciones según su similitud.**

Es de tipo no-supervisado (no requiere etiquetas para entrenar).

![](./resources/clustering.png)

### DBScan

DBscan es un algoritmo de clustering basado en densidad. Su funcionamiento se basa en clasificar los datos en tres categorías:

- **Core point**: Es un punto que contiene un número minimo de vecinos cerca de un vecindario (esfera de radio ɛ)

- **Border point**: Es un punto que no es core, pero que es alcanzable por un Core Point. Es decir, existe un camino entre el Core Point y este.

- **Outlier**: Es un punto que no es core point y a la vez, que no tiene un camino entre un Core Point u Border Point.

<div style="text-align: center;">
    <img src="../../recursos/2023-01/16-Detección-anomalias/dbscan.png" style="width: 50%;">
</div>

In [ ]:
from sklearn.cluster import DBSCAN

# El parámetro eps indicará el tamaño de la esfera y por tanto, la cantidad de outliers
# Mientras menor sea el tamaño de eps, mayor la cantidad de outliers
clustering = DBSCAN(eps=0.3, min_samples=3).fit(scaled_features)

db_scan_labels = clustering.labels_

db_scan_labels

In [ ]:
db_scan_labels.max()

In [ ]:
# Nota: Los labels de los outliers son -1
df[db_scan_labels == -1]

In [ ]:
# Es más sencillo visualizar los outliers con color=db_scan_labels == -1
px.scatter(df, x="x_umap", y="y_umap", color=db_scan_labels == -1)

In [ ]:
px.scatter(df, x="x_umap", y="y_umap", color=db_scan_labels.astype(str))

In [ ]:
from sklearn.cluster import DBSCAN

# Volvemos a hacer lo mismo pero con las dimensiones de UMAP
clustering_projections = DBSCAN(eps=0.3, min_samples=3).fit(wine_features_umap_embedded)

db_scan_labels = clustering_projections.labels_

db_scan_labels

In [ ]:
wine_features_umap_embedded

In [ ]:
px.scatter(df, x="x_umap", y="y_umap", color=db_scan_labels.astype(str))

## Métodos Específicos Provistos por Scikit-Learn

<div style="text-align: center;">
    <img src="../../recursos/2023-01/16-Detección-anomalias/outlier_methods_sklearn.png" style="width: 50%;">
</div>

### Isolation Forest

El concepto subyacente de Isolation Forest es que las anomalías son datos que son pocos y diferentes, y por lo tanto, **más fáciles de aislar** que los datos normales. Isolar se refiere al proceso de separar un punto de datos del resto del conjunto de datos.

#### ¿Cómo funciona?

1. Construcción de Árboles de Aislamiento: Isolation Forest construye múltiples árboles de decisión, llamados árboles de aislamiento, para aislar cada muestra de datos. Para cada árbol, el proceso es como sigue:
    - **Seleccionar aleatoriamente una variable** y luego selecciona un valor aleatorio de corte entre el mínimo y máximo valor de la característica seleccionada.
    - **Dividir el conjunto de datos en dos subconjuntos**: uno con valores menores que el valor de corte y otro con valores mayores o iguales.
    - **Repetir este proceso** recursivamente en cada subconjunto hasta que todos los datos han sido aislados o se alcanza un límite predeterminado en la profundidad del árbol.
2. Propiedad de Aislamiento: En teoría, las anomalías, al ser pocos y distintos, **requieren menos divisiones** aleatorias para ser aislados en comparación con los puntos de datos normales. Por lo tanto, los puntos de datos que tienen caminos cortos en los árboles son más propensos a ser anomalías.

#### Cálculo de puntuación de Anomalía

Una vez construidos los árboles, Isolation Forest evalúa la anormalidad de los datos utilizando una puntuación de anomalía basada en la **longitud del camino promedio** de un punto de datos en los árboles de aislamiento.
**Cuanto más corto es el camino promedio de un punto de datos, más alta es la puntuación de anomalía**, lo que indica una mayor probabilidad de ser una anomalía.

<div style="text-align: center;">
    <img src="../../recursos/2023-01/16-Detección-anomalias/itree.png" style="width: 50%;">
</div>
    
<center>Fuente:https://betterprogramming.pub/anomaly-detection-with-isolation-forest-e41f1f55cc6</center>

#### Ventajas de Isolation Forest
- **Eficiencia**: Es eficaz en grandes conjuntos de datos y con una alta dimensionalidad de datos.
- **No necesita normalización**: A diferencia de otros métodos que requieren normalización de datos, Isolation Forest no necesita normalizar los datos porque el método de partición es independiente de la distribución.
- **Escalabilidad y Facilidad de Implementación**: Es relativamente fácil de implementar y escalar en comparación con otros métodos de detección de anomalías.


<center>Trafico web a través del tiempo<center/>

<div style="text-align: center;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/b/b9/Anomalous_Web_Traffic.png" style="width: 60%;">
</div>


In [ ]:
import numpy as np
from sklearn.ensemble import IsolationForest

isf = IsolationForest(n_estimators=20, random_state=42)
outliers = isf.fit_predict(scaled_features)

In [ ]:
px.scatter(df, x="x_umap", y="y_umap", color=outliers.astype(str))

#### Novelty Detection

Ya que tenemos el modelo entrenado, podemos aplicarlo para clasificar una nueva observación (fuera del conjunto de entrenamiento) como outlier o no. Podemos lograr esto a través del método `.predict`:

In [ ]:
row = scaled_features[100, :].copy() # obtenemos una fila
row

In [ ]:
isf.predict([row]) # 1: no es outlier

In [ ]:
row[0] *= -100 # cambiar feature 0  * -1000 
row[5] *= 500 # cambiar feature 5 * 500
row[10] *= 999 # cambiar feature 10 * 999
row

In [ ]:
isf.predict([row]) # -1: es outlier